In [1]:
!pip install torch torchvision torchaudio

In [2]:
!pip install opacus

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.9/239.9 kB 9.0 MB/s eta 0:00:00


In [3]:
import warnings
warnings.simplefilter("ignore")

#MAX_GRAD_NORM = 1.2
EPSILON = 3.0
DELTA = 1e-5
EPOCHS = 25

#Adaptive Clipping
initial_clip_norm = 1.0
eta_C = 0.2  # Learning rate for updating the clipping norm
target_quantile = 0.5  # Target quantile, e.g., 0.5 for the median

LR = 0.5

In [4]:
BATCH_SIZE = 2048
MAX_PHYSICAL_BATCH_SIZE = 128

In [5]:
import torch
import torchvision
import torchvision.transforms as transforms

# These values, specific to the CIFAR10 dataset, are assumed to be known.
# If necessary, they can be computed with modest privacy budgets.
CIFAR10_MEAN = (0.4914, 0.4822, 0.4465)
CIFAR10_STD_DEV = (0.2023, 0.1994, 0.2010)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(CIFAR10_MEAN, CIFAR10_STD_DEV),
])

In [6]:
from torchvision.datasets import CIFAR10

DATA_ROOT = '../cifar10'

train_dataset = CIFAR10(
    root=DATA_ROOT, train=True, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
)

test_dataset = CIFAR10(
    root=DATA_ROOT, train=False, download=True, transform=transform)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
)

100%|██████████| 170M/170M [00:05<00:00, 30.8MB/s]


Extracting ../cifar10/cifar-10-python.tar.gz to ../cifar10
Files already downloaded and verified


### **ResNet-n from torch**

In [ ]:
from torchvision import models

### **ResNet20 (with Adaptive Clipping)**

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from opacus import PrivacyEngine

# Weight Standardized Convolution Layer
class WSConv2d(nn.Conv2d):
    def forward(self, x):
        # Weight standardization
        mean = self.weight.mean(dim=[1, 2, 3], keepdim=True)
        std = self.weight.std(dim=[1, 2, 3], keepdim=True)
        weight_standardized = (self.weight - mean) / (std + 1e-5)
        return F.conv2d(x, weight_standardized, self.bias, self.stride,
                        self.padding, self.dilation, self.groups)

# Basic Block for WRN-16-4
class BasicBlockWS(nn.Module):
    def __init__(self, in_planes, out_planes, stride, num_groups, dropRate=0.0):
        super(BasicBlockWS, self).__init__()
        self.gn1 = nn.GroupNorm(num_groups, in_planes)  # GroupNorm
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = WSConv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.gn2 = nn.GroupNorm(num_groups, out_planes)  # GroupNorm
        self.conv2 = WSConv2d(out_planes, out_planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.dropRate = dropRate

        # Shortcut connection
        self.equalInOut = (in_planes == out_planes)
        self.shortcut = (not self.equalInOut) and WSConv2d(in_planes, out_planes, kernel_size=1, stride=stride, padding=0, bias=False) or None

    def forward(self, x):
        if not self.equalInOut:
            x = self.relu(self.gn1(x))
        out = self.relu(self.gn1(x if self.equalInOut else x))
        out = self.conv1(out)
        out = self.gn2(out)
        if self.dropRate > 0:
            out = F.dropout(out, p=self.dropRate, training=self.training)
        out = self.conv2(out)
        return torch.add(x if self.equalInOut else self.shortcut(x), out)

# WRN-16-4 with GroupNorm
class WideResNetWS(nn.Module):
    def __init__(self, depth, widen_factor, num_classes=10, dropRate=0.0):
        super(WideResNetWS, self).__init__()
        nChannels = [16, 16*widen_factor, 32*widen_factor, 64*widen_factor]
        assert ((depth - 4) % 6 == 0)
        n = (depth - 4) // 6
        block = BasicBlockWS
        self.in_channels = 16

        # Initial convolutional layer
        self.conv1 = WSConv2d(3, self.in_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.gn1 = nn.GroupNorm(4, self.in_channels)  # GroupNorm for DP compatibility

        # Residual blocks with GroupNorm
        self.layer1 = self._make_layer(block, nChannels[0], nChannels[1], n, stride=1, num_groups=4, dropRate=dropRate)
        self.layer2 = self._make_layer(block, nChannels[1], nChannels[2], n, stride=2, num_groups=8, dropRate=dropRate)
        self.layer3 = self._make_layer(block, nChannels[2], nChannels[3], n, stride=2, num_groups=16, dropRate=dropRate)

        # Fully connected layer
        self.fc = nn.Linear(nChannels[3], num_classes)

    def _make_layer(self, block, in_planes, out_planes, num_blocks, stride, num_groups, dropRate):
        layers = []
        for _ in range(num_blocks):
            layers.append(block(in_planes, out_planes, stride, num_groups=num_groups, dropRate=dropRate))
            in_planes = out_planes
            stride = 1  # Only the first block has a stride > 1
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.gn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out




### Model Assignment

In [9]:
# Instantiate WRN-16-4
model = WideResNetWS(depth=16, widen_factor=4, num_classes=10)

In [10]:
from opacus.validators import ModuleValidator

errors = ModuleValidator.validate(model, strict=False)
errors[-5:]

[]

In [11]:
model = ModuleValidator.fix(model)
ModuleValidator.validate(model, strict=False)

[]

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = model.to(device)

In [13]:
import torch.nn as nn
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.9)

In [14]:
def accuracy(preds, labels):
    return (preds == labels).mean()

In [15]:
from opacus import PrivacyEngine

privacy_engine = PrivacyEngine()

model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimizer,
    data_loader=train_loader,
    epochs=EPOCHS,
    target_epsilon=EPSILON,
    target_delta=DELTA,
    max_grad_norm=1.0,  #Starting with grad norm as 1 and adjusting it during training according to the specific quantile of the gradient norm distribution
)

print(f"Using sigma={optimizer.noise_multiplier} and C=1.0")

Using sigma=1.61376953125 and C=1.0


In [16]:
def adaptive_clipping(grad_norms, clip_norm, target_quantile, eta_C):   #Adjusts the clipping norm using geometric updates to match a target quantile.
    #Calculate the fraction of gradients below the current clip norm
    clipped_fraction = np.mean(grad_norms <= clip_norm)

    #Update the clipping norm
    new_clip_norm = clip_norm * np.exp(-eta_C * (clipped_fraction - target_quantile))
    return new_clip_norm

def calculate_grad_norms(model):    #Calculates the L2 norm of gradients for each parameter in the model.
    grad_norms = []
    for p in model.parameters():
        if p.grad is not None:
            grad_norms.append(p.grad.data.norm(2).item())
    return np.array(grad_norms)

In [17]:
import numpy as np
from opacus.utils.batch_memory_manager import BatchMemoryManager

def train(model, train_loader, optimizer, epoch, device):
    model.train()
    criterion = nn.CrossEntropyLoss()

    losses = []
    top1_acc = []
    clip_norm = initial_clip_norm  #Start with an initial clip norm

    with BatchMemoryManager(
        data_loader=train_loader,
        max_physical_batch_size=MAX_PHYSICAL_BATCH_SIZE,
        optimizer=optimizer
    ) as memory_safe_data_loader:

        for i, (images, target) in enumerate(memory_safe_data_loader):
            optimizer.zero_grad()
            images = images.to(device)
            target = target.to(device)

            #compute output
            output = model(images)
            loss = criterion(output, target)

            preds = np.argmax(output.detach().cpu().numpy(), axis=1)
            labels = target.detach().cpu().numpy()

            #measure accuracy and record loss
            acc = accuracy(preds, labels)

            losses.append(loss.item())
            top1_acc.append(acc)

            loss.backward()

            #Adaptive Clipping
            #Calculate gradient norms for the current batch
            grad_norms = calculate_grad_norms(model)
            #Update the clipping norm based on gradient norms
            clip_norm = adaptive_clipping(grad_norms, clip_norm, target_quantile, eta_C)
            #Clip gradients to the updated norm
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip_norm)

            #Grouped Gradient Clipping
            '''
            clip_gradients_by_group(model, grouped_clipping_norms)
            '''

            optimizer.step()

            if (i+1) % 200 == 0:
                epsilon = privacy_engine.get_epsilon(DELTA)
                print(
                    f"\tTrain Epoch: {epoch} \t"
                    f"Loss: {np.mean(losses):.6f} "
                    f"Acc@1: {np.mean(top1_acc) * 100:.6f} "
                    f"(ε = {epsilon:.2f}, δ = {DELTA})"
                )
    mean_loss = np.mean(losses)
    mean_acc = np.mean(top1_acc)
    return mean_loss, mean_acc

In [18]:
def test(model, test_loader, device):
    model.eval()
    criterion = nn.CrossEntropyLoss()
    losses = []
    top1_acc = []

    with torch.no_grad():
        for images, target in test_loader:
            images = images.to(device)
            target = target.to(device)

            output = model(images)
            loss = criterion(output, target)
            preds = np.argmax(output.detach().cpu().numpy(), axis=1)
            labels = target.detach().cpu().numpy()
            acc = accuracy(preds, labels)

            losses.append(loss.item())
            top1_acc.append(acc)

    top1_avg = np.mean(top1_acc)

    print(
        f"\tTest set:"
        f"Loss: {np.mean(losses):.6f} "
        f"Acc: {top1_avg * 100:.6f} "
    )
    # return np.mean(top1_acc)
    mean_loss = np.mean(losses)
    mean_acc = np.mean(top1_acc)
    return mean_loss, mean_acc

In [19]:
from tqdm.notebook import tqdm

for epoch in tqdm(range(EPOCHS), desc="Epoch", unit="epoch"):
    train(model, train_loader, optimizer, epoch + 1, device)

Epoch:   0%|          | 0/25 [00:00<?, ?epoch/s]

	Train Epoch: 1 	Loss: 7.379809 Acc@1: 20.297936 (ε = 0.49, δ = 1e-05)
	Train Epoch: 2 	Loss: 3.289516 Acc@1: 38.525541 (ε = 0.75, δ = 1e-05)
	Train Epoch: 2 	Loss: 3.259066 Acc@1: 39.906320 (ε = 0.86, δ = 1e-05)
	Train Epoch: 3 	Loss: 3.225053 Acc@1: 42.272590 (ε = 0.95, δ = 1e-05)
	Train Epoch: 3 	Loss: 3.233252 Acc@1: 42.888495 (ε = 1.03, δ = 1e-05)
	Train Epoch: 4 	Loss: 3.176733 Acc@1: 44.110448 (ε = 1.11, δ = 1e-05)
	Train Epoch: 4 	Loss: 3.113922 Acc@1: 45.339192 (ε = 1.18, δ = 1e-05)
	Train Epoch: 5 	Loss: 3.082169 Acc@1: 47.275910 (ε = 1.25, δ = 1e-05)
	Train Epoch: 5 	Loss: 3.084711 Acc@1: 48.000198 (ε = 1.31, δ = 1e-05)
	Train Epoch: 6 	Loss: 3.024997 Acc@1: 49.146163 (ε = 1.38, δ = 1e-05)
	Train Epoch: 6 	Loss: 3.046153 Acc@1: 49.375294 (ε = 1.43, δ = 1e-05)
	Train Epoch: 7 	Loss: 2.899478 Acc@1: 50.203543 (ε = 1.49, δ = 1e-05)
	Train Epoch: 7 	Loss: 2.988860 Acc@1: 50.010574 (ε = 1.55, δ = 1e-05)
	Train Epoch: 8 	Loss: 2.974524 Acc@1: 50.929979 (ε = 1.60, δ = 1e-05)
	Train

In [20]:
top1_acc = test(model, test_loader, device)

	Test set:Loss: 3.095474 Acc: 55.610740 
